# Analyzing CIA Factbook Data Using SQL

## 1. Introduction :

In this project we work with [CIA World Factbook](https://github.com/fivethirtyeight/data/blob/master/college-majors/recent-grads.csv) data. It is a compendium of statistics about all of the countries in Earth. The Factbook contains demographic information like population, population_growth, area etc.

** Connecting the jupyter notebook to database :**

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

'Connected: None@factbook.db'

### 2. Overview of the Data :

#### 2.1 Displaying contents in the database :

- Here we display different tables and corresponding details in the database

In [2]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


#### 2.2 Display factsbook table :

Here we displayed first five rows in the facts table. Description of the columns :

- name - The name of the country.
- area - The total land and sea area of the country.
- population - The country's population.
- population_growth- The country's population growth as a  percentage.
- birth_rate - The country's birth rate, or the number of births a year per 1,000 people.
- death_rate - The country's death rate, or the number of death a year per 1,000 people.
- area- The country's total area (both land and water).
- area_land - The country's land area in square kilometers.
- area_water - The country's waterarea in square kilometers.

In [3]:
%%sql
SELECT * FROM facts limit 5;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


### 3. Summary Statistics :

Displaying following values from the database in a single query.

- min_population - minimum population
- max_population - maximum population
- min_pop_growth - minimum population growth
- max_pop_growth - maximum population growth

In [4]:
%%sql
SELECT MIN(population) as min_population,
MAX(population) as max_population,
MIN(population_growth) as min_pop_growth, 
MAX(population_growth) as max_pop_growth FROM facts;

Done.


min_population,max_population,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


### 4. Exploring Outliers :

#### 4.1 Retrieving Country with minimum population :

In [5]:
%%sql
SELECT * FROM facts WHERE population=(SELECT MIN(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


#### 4.2 Retrieving Country with maximum population :

In [6]:
%%sql
SELECT * FROM facts WHERE population=(SELECT MAX(population) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


### 5. Exploring Average Population and Area :

Calculating the average population and average area in the facts table.

In [7]:
%%sql
SELECT AVG(population), AVG(area) FROM facts;

Done.


AVG(population),AVG(area)
62094928.32231405,555093.546184739


### 6. Finding Densely Populated Countries :

Finding the countries have above average population and below average area.

In [8]:
%%sql
SELECT * FROM facts 
WHERE 
population > (SELECT AVG(population) FROM facts)
AND
area < (SELECT AVG(area) FROM facts)

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


### 7. Optional Questions :

#### Countries having the highest ratios of water to land :

In [9]:
%%sql
SELECT *, CAST(area_water as float)/CAST(area_land as float) water_to_land_ratio FROM facts
WHERE water_to_land_ratio = 
(SELECT 
MAX(CAST(area_water as float)/CAST(area_land as float))
FROM facts);

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,water_to_land_ratio
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None,905.6666666666666


#### Which countries have more water than land?

In [10]:
%%sql
SELECT * FROM facts 
WHERE 
area_water > area_land

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


#### Which countries will add the most people to their population next year?

- Displaying countries having more than 3% population growth by next year.

In [11]:
%%sql
SELECT * FROM facts
WHERE
population_growth > 3;

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
27,uv,Burkina Faso,274200,273800,400,18931686,3.03,42.03,11.72,0.0
29,by,Burundi,27830,25680,2150,10742276,3.28,42.01,9.27,0.0
106,mi,Malawi,118484,94080,24404,17964697,3.32,41.56,8.41,0.0
128,ng,Niger,None,1266700,300,18045729,3.25,45.45,12.42,0.56
141,qa,Qatar,11586,11586,0,2194817,3.07,9.84,1.53,22.39
162,od,South Sudan,644329,None,None,12042910,4.02,36.91,8.18,11.47
182,ug,Uganda,241038,197100,43938,37101745,3.24,43.79,10.69,0.74


#### Which countries have a higher death rate than birth rate?

In [13]:
%%sql
SELECT * FROM facts 
WHERE
death_rate>birth_rate

Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
